<a href="https://colab.research.google.com/github/joangog/object-detection/blob/main/mask_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Model training on mask dataset

<!-- The following models will be trained with their respective hyperparameters:

MASKD:

| Model | Backbone | Batch Size | Epochs | LR Scheduler | Learning Rate | Optimizer | Momentum | Weight Decay |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| YOLOv5s |  Custom | 16 | 100 | CycleLR | 0.01 | SGD | 0.937 | 0.0005
| YOLOv5s |  Custom | 16 | 100 | CycleLR | 0.001 | Adam | 0.937 | 0.0005
| YOLOv5m |  Custom | 16 | 100 | CycleLR | 0.01 | SGD | 0.937 | 0.0005
| YOLOv5l |  Custom | 16 |  |  |  |
| YOLOv3-tiny |  Darknet53 | 16 | 100 | CycleLR | 0.01 | SGD | 0.937 | 0.0005

<br> -->


PWMFD:

| Model | Backbone | Batch Size | Epochs | LR Scheduler | Learning Rate | Optimizer | Momentum | Weight Decay |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| YOLOv5s |  Custom | 32 | 100 | CycleLR | 0.01 | SGD | 0.937 | 0.0005
| YOLOv3-tiny |  Darknet53 | 32 | 100 | CycleLR | 0.01 | SGD | 0.937 | 0.0005



The following datasets can be used:
*   MASKD (AICrowd)
*   Properly Wearing Masked Face Detection Dataset 

<br>

**Note: GPU Runtime needed (hosted or local)**

*Example GPU: Tesla K80, 460.32.03, 11441 MiB*




In [ ]:
# Show system specs
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla K80, 460.32.03, 11441 MiB


### Initialization


In [ ]:
# Parameters

dataset_name = 'PWMFD'  # 'MASKD' or 'PWMFD'

load_ckpt = False  # Flag for whether to load existing checkpoint or not
load_cfg = False  # Flag for whether to load custom model configs or not
pretrained = True  # (If load_ckpt == False) Flag for whether to start from COCO-pretrained model or randomly init weights
data_augment = False  # Flag for whether to use albumentations data augmentations

img_res = 320  # Image resolution (for YOLO models), try 640, 320 or 160
num_workers = 2  #  Data loader workers
batch_size = 32  # Data loader batch size (both training and inference)

num_epochs = 50  # Epochs

lr = 0.01  # Learning rate (initial)
step_size = 3  # Learning rate step size (only for StepLR, YOLO uses CycleLR)
gamma = 0.1  # Learning rate decay (only for StepLR)

optimizer_type = 'sgd'  # 'sgd' or 'adam'
momentum = 0.937  # Optimizer momentum (only for SGD optimizer)
weight_decay = 0.0005  # Optimizer weight decay

# Directories

load_ckpt_path = '/content/drive/MyDrive/object-detection-checkpoints/PWMFD_yolov5s_1637236944_run/weights/last.pt'  # Loaded weights path
save_ckpt_dir = '/content/drive/MyDrive/object-detection-checkpoints'  # Model save root directory (used when load_ckpt is False)
load_cfg_path  = '/content/config/yolov5s-tr-exp.yaml'

import os
root_dir = os.getcwd()  # Root dir of project
dataset_dir = os.path.join(root_dir,f'dataset_{dataset_name}')

img_dir = os.path.join(dataset_dir,'images')
val_img_dir = os.path.join(img_dir,'val_images')
train_img_dir = os.path.join(img_dir,'train_images')

label_dir = os.path.join(dataset_dir,'labels')
val_label_dir = os.path.join(label_dir,'val_images')
train_label_dir = os.path.join(label_dir,'train_images')

ann_dir = os.path.join(dataset_dir,'annotations')

### Get requirements
*Note: Restart runtime after installation*

In [ ]:
# Clone asset files
!cd {root_dir}
!git clone https://github.com/joangog/object-detection-assets
!mv -n {os.path.join(root_dir,'object-detection-assets','scripts')} ./
!mv -n {os.path.join(root_dir,'object-detection-assets','config')} ./
!mv -n {os.path.join(root_dir,'object-detection-assets','requirements.txt')} ./
!rm -rf {os.path.join(root_dir,'object-detection-assets')}


Cloning into 'object-detection-assets'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 175 (delta 67), reused 139 (delta 34), pack-reused 0
Receiving objects: 100% (175/175), 35.59 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [ ]:
# Install packages
!cd {root_dir}
if data_augment:
  !pip install albumentations==1.0.3
else:
  !pip install albumentations==0.1.12
!pip install -r requirements.txt

     |████████████████████████████████| 631 kB 5.2 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654020 sha256=7e017afbf75d6334849c353cf204b2b461a5cc8566dc8a832e68411851bcefb2
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Cloning https://github.com/googlecolab/colabtools.git to /tmp/pip-req-build-zgmgd9aw
  Running command git clone -q https://github.com/googlecolab/colabtools.git /tmp/pip-req-build-zgmgd9aw
     |████████████████████████████████| 26.5 MB 46 kB/s 
     |████████████████████████████████| 7.1 MB 32.2 MB/s 
     |████████████████████████████████| 5.6 MB 38.5 MB/s 
     |████████████████████████████████| 458.3 MB 11 kB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
     |███████████████████

In [ ]:
# Install Yolov5
!cd {root_dir}
!git clone https://github.com/joangog/yolov5
!pip install -r {os.path.join(root_dir,'yolov5','requirements.txt')}

# Replace albumentation augmentations with custom ones
!cd utils
!rm augmentations.py
!cp {os.path.join(root_dir,'scripts','augmentations.py')} {os.path.join(root_dir,'yolov5','utils','augmentations.py')} 

Cloning into 'yolov5'...
remote: Enumerating objects: 9899, done.
remote: Total 9899 (delta 0), reused 0 (delta 0), pack-reused 9899
Receiving objects: 100% (9899/9899), 10.29 MiB | 21.86 MiB/s, done.
Resolving deltas: 100% (6861/6861), done.
     |████████████████████████████████| 596 kB 5.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
/bin/bash: line 0: cd: utils: No such file or directory
rm: cannot remove 'augmentations.py': No such file or directory
cp: cannot stat '/content/scripts/augmentations.py': No such file or directory


In [ ]:
# Install Yolov3
!cd {root_dir}
!git clone https://github.com/ultralytics/yolov3
!pip install -r {os.path.join(root_dir,'yolov3','requirements.txt')}

Cloning into 'yolov3'...
remote: Enumerating objects: 9946, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 9946 (delta 15), reused 27 (delta 5), pack-reused 9858
Receiving objects: 100% (9946/9946), 9.37 MiB | 21.18 MiB/s, done.
Resolving deltas: 100% (6680/6680), done.


In [ ]:
# Install unrar command
if os.geteuid() != 0:  # If not root, ask for sudo priviledges
  from getpass import getpass
  password = getpass('Insert sudo password:')
  !echo {password} | sudo -S -k apt-get install unrar
else:
  !apt-get install unrar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unrar is already the newest version (1:5.5.8-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


### Import packages

In [ ]:
# Comment google.colab packages if using local runtime
from google.colab import files
from google.colab import drive

import os, sys
import math
import time
import copy
import re

import numpy as np
import pandas as pd
import json
import xml.etree.ElementTree as ET
import PIL
import IPython

import matplotlib
import matplotlib.pyplot as plt

import torch
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.models.detection as M
import torchvision.transforms.functional as F
import torchvision.utils as U
from torchvision.datasets import CocoDetection

from pycocotools import coco
from pycocotools import mask as cocomask

from ptflops import get_model_complexity_info

import scripts.utils as SU
import scripts.transforms as ST
import scripts.engine as SE
import scripts.coco_utils as SCU
from scripts.coco_eval import CocoEvaluator

### Define auxiliary functions

In [ ]:
def add_motion_blur(img, kernel_size, kernel_angle):
  
  kernel = np.zeros((kernel_size, kernel_size))
  
  if kernel_angle == 'v':  # Vertical Motion Blur
    kernel[:, int((kernel_size - 1)/2)] = np.ones(kernel_size)

  elif kernel_angle == 'h':  # Horizontal Motion Blur
    kernel[int((kernel_size - 1)/2), :] = np.ones(kernel_size)

  return cv2.filter2D(img, -1, kernel)

### (Optional) Connect to GDrive for storage access
*Note: Not possible with local runtime*

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Download Mask dataset

In [ ]:
!cd {root_dir}
!mkdir -p dataset_{dataset_name}
!cd {dataset_dir}
!mkdir -p {img_dir} {ann_dir} {label_dir}
!cd {img_dir}
!mkdir {val_img_dir} {train_img_dir}
!cd {label_dir}
!mkdir {val_label_dir} {train_label_dir}

In [ ]:
if dataset_name == 'MASKD':

  !cd {root_dir}

  # Download validation images
  if not os.path.exists('val_images.zip'):
    !gdown --id '101F2k6PJ-tD_uwlsCG7zzGF9ILJW01M1'
  !unzip -q -n 'val_images.zip' -d {img_dir}

  # Download train images
  if not os.path.exists('train_images.zip'):
    !gdown --id '1vD_Sxg7dHkB_8OJLsHngBWvp5iGJAETQ'
  !unzip -q -n 'train_images.zip' -d {img_dir}

  # Download validation annotations
  if not os.path.exists('val.json'):
    !gdown -O {os.path.join(ann_dir,'val.json')} --id '1YLV7-7vmiNdFI8Xpdx_jbhnxfgQRWrgF'

  # Download train annotations
  if not os.path.exists('train.json'):
    !gdown -O {os.path.join(ann_dir,'train.json')}  --id '1AqeDJps-aZ743vFJ6p2_RjtSFjPtIOtD'

elif dataset_name == 'PWMFD':

  # Download validation images
  if not os.path.exists('val_images.rar'):
    !gdown -O 'val_images.rar' --id  1ZXuSwoRvTnnca81RUj3kMoLFZJ6auAwT
  !unrar e -idq -o- 'val_images.rar' -d {val_img_dir}

  # Download train images
  if not os.path.exists('train_images.rar'):
    !gdown -O 'train_images.rar' --id  16uI5ZEiq2JEYBH4_DhmxVAHSqzY5UQ1b
  !unrar e -idq -o- 'train_images.rar' -d {train_img_dir} 

  # Convert annotation files from PASCAL VOC .xml to COCO .json (only for PWMFD dataset)
  
  label_ids = {'with_mask': 1, 'without_mask': 2, 'incorrect_mask': 3}  # BG class is 0
    
  for type_img in ['val', 'train']:

    ann_count = 0  # Annotation counter

    images = []
    categories = []
    annotations = []

    xml_files = os.listdir(os.path.join(os.path.join(img_dir,f'{type_img}_images')))
    xml_files = [file for file in xml_files if '.xml' in file]

    # Categories
    for label in label_ids:
      categories.append(
          {
            'supercategory': 'none',
            'id': label_ids[label],
            'name': label
          }
      )

    for xml_file in xml_files:  # For each annotation file

      tree = ET.parse(os.path.join(os.path.join(img_dir,f'{type_img}_images'),xml_file))
      root = tree.getroot()

      # Image
      file_name = root[0].text
      height = int(root[1][1].text)
      width = int(root[1][0].text)
      id = int(re.sub(r'^\D*0*', '', file_name).replace('.jpg',''))
      images.append(
          {
            'file_name': file_name,
            'height': height,
            'width': width,
            'id': id
          }
      )
      
      # Annotations
      if len(root) > 2:  # If annotations (object attribute) exist, they will be after the 1-index attribute in the XML
        for i in range(2,len(root)):
          category_id = label_ids[root[i][0].text]
          xmin = int(root[i][1][0].text)
          ymin = int(root[i][1][1].text)
          xmax = int(root[i][1][2].text)
          ymax = int(root[i][1][3].text)
          annotations.append(
              {
                'iscrowd': 0,
                'image_id': id,
                'bbox': [xmin, ymin, xmax-xmin, ymax-ymin],
                'area': (xmax-xmin) * (ymax-ymin),
                'category_id': category_id,
                'ignore': 0,
                'id': ann_count,
                'segmentation': []
              }
          )
          ann_count += 1

    coco_dict = {
    'info': {},
    'images': images,
    'categories': categories,
    'annotations': annotations,
    'licenses': []
    }

    with open(os.path.join(ann_dir,f'{type_img}.json'),'w') as outfile:
      json.dump(coco_dict, outfile, indent=3)


# Copy COCO annotations in images folder
!cp {os.path.join(ann_dir,'val.json')} {val_img_dir}
!cp {os.path.join(ann_dir,'train.json')} {train_img_dir}

Downloading...
From: https://drive.google.com/uc?id=1ZXuSwoRvTnnca81RUj3kMoLFZJ6auAwT
To: /content/val_images.rar
100% 204M/204M [00:01<00:00, 104MB/s]
Downloading...
From: https://drive.google.com/uc?id=16uI5ZEiq2JEYBH4_DhmxVAHSqzY5UQ1b
To: /content/train_images.rar
100% 1.13G/1.13G [00:10<00:00, 109MB/s] 


### (Only for YOLO models) Convert format of annotations from COCO to YOLO

In [ ]:
for type_images in ['val', 'train']:

    ann_path = os.path.join(ann_dir, type_images + ".json")
    dataset = coco.COCO(ann_path)
    img_ids = dataset.getImgIds()

    for img_id in img_ids:

      img_anns = dataset.loadAnns(dataset.getAnnIds([img_id]))
      img_data = dataset.loadImgs([img_id])[0]
      img_file = img_data['file_name']
      img_width = img_data['width']
      img_height = img_data['height']

      label_file = img_file.replace('.jpg','.txt')
      
      with open(os.path.join(label_dir, f'{type_images}_images', label_file), 'w') as outfile:
        for ann in img_anns:
          x_center = (ann['bbox'][0] + ann['bbox'][2]/2) / img_width  # convert x_min to x_center and normalize to [0,1]
          y_center = (ann['bbox'][1] + ann['bbox'][3]/2) / img_height  # convert y_min to y_center and normalize to [0,1]
          width = ann['bbox'][2] / img_width
          height = ann['bbox'][3] / img_height 
          outfile.write("{} {} {} {} {}\n".format(int(ann['category_id'])-1,x_center,y_center,width,height))   # Category ids must be 0-indexed


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


### Load Mask dataset

In [ ]:
val_ann_file = 'val.json'  # annotations
val_ann_path = os.path.join(val_img_dir,val_ann_file)  

train_ann_file = 'train.json'  # annotations
train_ann_path = os.path.join(train_img_dir,train_ann_file)  

# Define data transforms
transforms = ST.Compose([ST.ToTensor()])

# Create datasets
val_dataset = CocoDetection(val_img_dir, val_ann_path, transforms = transforms)
train_dataset = CocoDetection(train_img_dir, train_ann_path, transforms = transforms)

# Create data loaders
val_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=SU.collate_fn)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=SU.collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


### Load pre-trained model

In [ ]:
!cd {root_dir}

# Delete utils package to reload it (if loaded), because YOLOv3 and YOLOv5 have
# the same name for it and it causes error
try:
  sys.modules.pop('utils')
except:
  pass

# @markdown Model Selection { display-mode: 'form', run: 'auto' }
model_name = 'YOLOv5s' # @param ['SSD300 VGG16', 'SSDlite320 MobileNetV3-Large', 'Faster R-CNN ResNet-50 FPN', 'Faster R-CNN MobileNetV3-Large FPN', 'Mask R-CNN ResNet-50 FPN', 'YOLOv5s', 'YOLOv5m', 'YOLOv5l', 'YOLOv3', 'YOLOv3-tiny', 'YOLOv3-spp']

# @markdown *Note: If you get the error "Cache may be out of date, try 'force_reload=True'" then restart runtime.*

if model_name == 'SSD300 VGG16':
  model_id = 'ssd300_vgg16'
  model = M.ssd300_vgg16(pretrained=True, progress=True)
  model_img_size = (3,300,300)
elif model_name == 'SSDlite320 MobileNetV3-Large':
  model_id = 'ssdlite320_mobilenet_v3_large'
  model = M.ssdlite320_mobilenet_v3_large(pretrained=True, progress=True)
  model_img_size = (3,320,320)
elif model_name == 'Faster R-CNN ResNet-50 FPN':
  model_id = 'fasterrcnn_resnet50_fpn'
  model = M.fasterrcnn_resnet50_fpn(pretrained=True, progress=True)
  model_img_size = (3,800,800) # COCO's 640x640 in upscaled to the model's minimum 800x800
elif model_name == 'Faster R-CNN MobileNetV3-Large FPN':
  model_id = 'fasterrcnn_mobilenet_v3_large_fpn'
  model = M.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True, progress=True)
  model_img_size = (3,800,800) 
elif model_name == 'Mask R-CNN ResNet-50 FPN':
  model_id = 'maskrcnn_resnet50_fpn'
  model = M.maskrcnn_resnet50_fpn(pretrained=True, progress=True)
  model_img_size = (3,800,800)
elif model_name == 'YOLOv5s':
  model_id = 'yolov5s'
  model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True)
  model_img_size = (3,img_res,img_res)
elif model_name == 'YOLOv5m':
  model_id = 'yolov5m'
  model = torch.hub.load('ultralytics/yolov5', 'yolov5m', force_reload=True)
  model_img_size = (3,img_res,img_res)
elif model_name == 'YOLOv5l':
  model_id = 'yolov5l'
  model = torch.hub.load('ultralytics/yolov5', 'yolov5l', force_reload=True)
  model_img_size = (3,img_res,img_res)
elif model_name == 'YOLOv3':
  model_id = 'yolov3'
  model = torch.hub.load('ultralytics/yolov3', 'yolov3', force_reload=True)
  model_img_size = (3,img_res,img_res)
elif model_name == 'YOLOv3-tiny':
  model_id = 'yolov3_tiny'
  model = torch.hub.load('ultralytics/yolov3', 'yolov3_tiny', force_reload=True)
  model_img_size = (3,img_res,img_res)
elif model_name == 'YOLOv3-spp':
  model_id = 'yolov3_spp'
  model = torch.hub.load('ultralytics/yolov3', 'yolov3_spp', force_reload=True)
  model_img_size = (3,img_res,img_res)

print('-------------------------------------------------------------------------------------------------------\n')

print(f'Loaded model: {model_name}')
model_params = round(sum([param.numel() for param in model.parameters()]) / 1000000, 1)
print(f'\t- Parameters: {model_params}M')
model_macs, _ = get_model_complexity_info(model, model_img_size, as_strings=False, 
                                          print_per_layer_stat=False, verbose=False)
model_gflops = round(2 * int(model_macs) / 1000000000, 1)
print(f'\t- GFLOPs: {model_gflops}')

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: torch>=1.7.0 not found and is required by YOLOv5, attempting auto-update...

requirements: torchvision>=0.8.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-11-20 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441MiB)




requirements: 3 packages updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



  0%|          | 0.00/14.0M [00:00<?, ?B/s]

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


-------------------------------------------------------------------------------------------------------

Loaded model: YOLOv5s
	- Parameters: 7.2M
	- GFLOPs: 4.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### (Optional) Open Tensorboard Monitor

In [ ]:
# Load Tensorboard
%reload_ext tensorboard
%tensorboard --bind_all --logdir {save_ckpt_dir}


<IPython.core.display.Javascript object>

### Train model

In [ ]:
# Set model checkpoint folder
start_time = int(time.time())
save_ckpt_folder = f'{dataset_name}_{model_id}_{start_time}_run'

# Prepare model for new dataset (for Fast R-CNN or Mask R-CNN)
if 'R-CNN' in model_name: 
  num_classes = len(val_dataset.coco.getCatIds())+1  # includes background (0) class
  # Get the number of input features for the bbox predictor
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # Replace the pre-trained head with a new one
  model.roi_heads.box_predictor = M.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
  if 'Mask R-CNN' in model_name:
    # Get the number of input features for the segmentation max predictor
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # Replace the mask predictor with a new one
    model.roi_heads.mask_predictor = M.mask_rcnn.MaskRCNNPredictor(in_features_mask, hidden_layer,num_classes)

# Model Training 

if 'YOLO' in model_name: # For YOLO models

  yolo_version = int(model_id[5])

  # Generate hyperparameter file
  %cp {root_dir}/yolov{yolo_version}/data/{'hyps/' if yolo_version == 5 else ''}hyp.scratch.yaml {root_dir}/hyp.yaml
  with open('hyp.yaml','a') as outfile:
    outfile.write(

f"""lr0: {lr}
momentum: {momentum}
weight_decay: {weight_decay}"""

)

  train_script_path = os.path.join(root_dir,f'yolov{yolo_version}','train.py')
  data_path = os.path.join(root_dir,'config',f'{dataset_name}_yolov{yolo_version}.yaml')

  if load_ckpt:  # If load a pre-existing checkpoint, resume from it
    temp_save_ckpt_folder = re.search(r'/' + dataset_name + '[^/]*/', load_ckpt_path).group(0).replace('/','')  # Get the save folder name from the loaded folder name
    temp_load_ckpt_path = load_ckpt_path
  else:  # If not load a pre-existing checkpoint, load pre-trained COCO weights or randomly initialize weights
    if pretrained:
      temp_save_ckpt_folder = save_ckpt_folder
      temp_load_ckpt_path = f'{model_id.replace("_","-")}.pt'
    else:
      temp_save_ckpt_folder = save_ckpt_folder
      temp_load_ckpt_path = '""'
  if load_cfg:  # If load custom model config
    temp_cfg_path = load_cfg_path
  else:
    temp_cfg_path = model_id.replace("_","-") + '.yaml'

  !python {train_script_path} --img {img_res} --epochs {num_epochs} --batch {batch_size} --project {save_ckpt_dir} --name {temp_save_ckpt_folder} --hyp "hyp.yaml" --data {data_path} \
  {'--resume ' + temp_load_ckpt_path if load_ckpt else '--weights ' + temp_load_ckpt_path} \
  {'--adam' if optimizer_type=='adam' else ''} \
  {'--cfg '  +  temp_cfg_path}

else: # For all the other models

  # Make checkpoint folder
  !mkdir {os.path.join(ckpt_dir,ckpt_folder)}

  # Load pre-existing checkpoint weights if required
  last_epoch = 0
  if load_ckpt:
    model.load_state_dict(torch.load(load_ckpt_path))
    last_epoch = int(re.search(r'epoch[0-9]+', load_ckpt_path).group(0)[5:])
    print(f'Loaded checkpoint: {load_ckpt_path}')

  # Get appropriate device for model
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  model.to(device)
  print(f'Pre-trained Model: {model_name}')

  # Construct an optimizer
  params = [p for p in model.parameters() if p.requires_grad]
  if optimizer_type == 'adam':
    optimizer = torch.optim.Adam(params, lr=lr, weight_decay=weight_decay)
  elif optimizer_type == 'sgd':
    optimizer = torch.optim.SGD(params, lr=lr, momentum=momentum, weight_decay=weight_decay)

  # Construct learning rate scheduler
  lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

  # Save training configs to .txt
  config_path = os.path.join(save_ckpt_dir,save_ckpt_folder,'cfg.txt')
  with open(config_path,'w') as outfile:
    outfile.write( 
                         
f"""dataset: {dataset_name}
model: {model_id}
timestamp: {start_time}
last_checkpoint: {load_ckpt_path if load_ckpt else None}
num_epochs: {num_epochs}
batch_size: {batch_size}
optimizer: {optimizer_type}
learning_rate: {lr}
step_size: {step_size}
gamma: {gamma}
momentum: {momentum}
weight_decay: {weight_decay}"""

)

  # Train model
  for epoch in range(num_epochs):

      curr_epoch = last_epoch + epoch + 1

      # Train for one epoch, printing every 10 iterations
      metric_logger = SE.train_one_epoch(model, optimizer, train_data_loader, device, epoch, print_freq=1)
      
      # Update the learning rate
      lr_scheduler.step()
      
      # Evaluate on the test dataset
      evaluator, _, _, _ = SE.evaluate(model, val_data_loader, device)

      # Save model checkpoint
      ckpt_file = f'epoch{curr_epoch}'
      ckpt_path = os.path.join(ckpt_dir,ckpt_folder,ckpt_file)
      torch.save(model.state_dict(), ckpt_path)

      # Delete previous model checkpoint
      previous_ckpt_file = f'epoch{curr_epoch-1}'
      if os.path.exists(previous_ckpt_file):
        os.remove(previous_ckpt_file)

train: weights=yolov5/yolov5s.pt, cfg=/content/config/yolov5s-tr-exp.yaml, data=/content/config/PWMFD_yolov5.yaml, hyp=hyp.yaml, epochs=50, batch_size=32, imgsz=320, rect=False, resume=/content/drive/MyDrive/object-detection-checkpoints/PWMFD_yolov5s_1637236944_run/weights/last.pt, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=/content/drive/MyDrive/object-detection-checkpoints, name=PWMFD_yolov5s_1637236944_run, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/joangog/yolov5 ✅
Resuming training from /content/drive/MyDrive/object-detection-checkpoints/PWMFD_yolov5s_1637236944_run/weights/last.pt
YOLOv5 🚀 v6.0-87-ge00b065 torch 1.9.0+cu102 CUDA:0 (Tesla K